In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import re, json
import requests
from bs4 import BeautifulSoup
from pandas import json_normalize

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
%matplotlib inline

import geopy.geocoders
from geopy.geocoders import ArcGIS
geopy.geocoders.options.default_timeout = None

import folium

from sklearn.cluster import KMeans

In [6]:
with open('data/nyu.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']

In [41]:
with open('data/brooklyn.geojson') as json_data:
    brooklyn_data = json.load(json_data)

In [9]:
def assemble_dataset():
    column_names = ['Neighborhood', 'Latitude', 'Longitude'] 
    neighborhoods = pd.DataFrame(columns=column_names)

    for data in neighborhoods_data:
        borough = neighborhood_name = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']

        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]

        if borough != 'Brooklyn': continue

        neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                              'Latitude': neighborhood_lat,
                                              'Longitude': neighborhood_lon}, ignore_index=True)
    return neighborhoods

In [11]:
brooklyn = assemble_dataset()
brooklyn.head()

,Neighborhood,Latitude,Longitude
0,Bay Ridge,40.625801,-74.030621
1,Bensonhurst,40.611009,-73.995180
2,Sunset Park,40.645103,-74.010316
3,Greenpoint,40.730201,-73.954241
4,Gravesend,40.595260,-73.973471


In [12]:
address = 'Brooklyn, NY'

geolocator = ArcGIS()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.692450000000065, -73.99035999999995.


In [40]:
map_brooklyn = folium.Map(location=[latitude-4.4e-2, longitude+4e-2], zoom_start=11.5) # Corrections added to better center the map

for lat, lng, label in zip(brooklyn['Latitude'], brooklyn['Longitude'], brooklyn['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

In [51]:
m = folium.Map(
    location=[latitude-4.4e-2, longitude+4e-2],
    zoom_start=11  # Limited levels of zoom for free Mapbox tiles.
)

folium.GeoJson(
    brooklyn_data,
    name='geojson'
).add_to(m)

# folium.LayerControl().add_to(m)
m

In [79]:
names = list(map(lambda x: x['properties']['name'], brooklyn_data['features']))

In [81]:
names.sort()

In [93]:
brooklyn_filtered = brooklyn.loc[brooklyn['Neighborhood'].isin(names)].reset_index()
brooklyn_filtered = brooklyn_filtered.drop(labels='index', axis=1)

In [94]:
brooklyn_filtered

,Neighborhood,Latitude,Longitude
0,Bay Ridge,40.625801,-74.030621
1,Bensonhurst,40.611009,-73.995180
2,Sunset Park,40.645103,-74.010316
3,Greenpoint,40.730201,-73.954241
4,Gravesend,40.595260,-73.973471
5,Brighton Beach,40.576825,-73.965094
6,Sheepshead Bay,40.586890,-73.943186
7,Flatbush,40.636326,-73.958401
8,Crown Heights,40.670829,-73.943291
9,East Flatbush,40.641718,-73.936103


In [108]:
m = folium.Map(
    location=[latitude-4.4e-2, longitude+4e-2],
    zoom_start=11  # Limited levels of zoom for free Mapbox tiles.
)

folium.Choropleth(
    geo_data='data/brooklyn.geojson',
    name='choropleth',
    data=brooklyn_filtered,
    columns=['Neighborhood', 'Longitude'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)

m